In [1]:
!pip -q install nninteractive SimpleITK huggingface_hub tifffile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 73.0 MB/s eta 0:

In [3]:
from google.colab import drive
drive.mount('/content/drive')
IMAGE_PATH = "/content/drive/MyDrive/CK7RY1~U.TIF"  # <- change this

Mounted at /content/drive


In [4]:
# === Cell 3: Multi-lumen membrane segmentation → per-slice "fill inside outer outline" (no circles) + 3D smooth ===
# Preserves lumen shape on each slice, fills the ring interior, supports multiple lumens, and is OOM-safe.

import os, time, shutil, gc, numpy as np, SimpleITK as sitk

from scipy.ndimage import (
    gaussian_filter, median_filter, grey_closing,
    binary_fill_holes, label
)
from skimage.morphology import (
    ball, disk, binary_opening, binary_closing,
    remove_small_objects, remove_small_holes
)

# ---------------- Knobs (tune as needed) ----------------
OUTPUT_DIR        = "/content/out_lumen3d"
OUT_TIF           = "lumen_mask_3d.tif"

# Preprocess
GAUSS_SIGMA       = (1.2, 1.2, 1.2)    # 3D Gaussian (X,Y,Z)
DESPECKLE_SIZE    = 3                  # 3×3×3 median
GREY_CLOSE_RAD    = 1                  # grayscale closing radius

# Threshold
THRESH_VAL_U8     = 26                # fixed 8-bit threshold for membrane channel

# Cleanup before CC
OPEN_RAD_3D       = 2                  # 3D opening after fill holes

# Keep lumens (3D)
MIN_SIZE_VOX      = 5000               # minimum 3D size (voxels) to keep a lumen

# Per-slice “fill inside outline”
SL_CLOSE_RAD      = 5                  # 2D closing radius before fill (smooth ragged ring)
SL_OPEN_RAD       = 0                  # optional 2D opening after fill (0 = skip)
SL_MIN_OBJECT_2D  = 400                # remove tiny 2D specks inside slice
SL_HOLE_AREA_2D   = 150_000             # fill interior holes up to this area on each slice

# Final 3D smoothing
FINAL_CLOSE_RAD   = 3                  # 3D closing (rounds/smooths across Z)
FINAL_OPEN_RAD    = 0                  # optional 3D opening after closing (0 = skip)

# --------------------------------------------------------

def log(*a): print("[LOG]", *a)

def to_itk_u8(xyz_u8):
    # (X,Y,Z) -> (Z,Y,X)
    return sitk.GetImageFromArray(np.transpose(xyz_u8, (2,1,0)))

t0 = time.time()
shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# (1) Load & 8-bit
img_itk = sitk.ReadImage(IMAGE_PATH)
arr_zyx = sitk.GetArrayFromImage(img_itk)                    # (Z,Y,X)
arr_xyz = np.transpose(arr_zyx, (2,1,0)).astype(np.float32)  # (X,Y,Z)
X, Y, Z = arr_xyz.shape
vmin, vmax = float(arr_xyz.min()), float(arr_xyz.max())
arr8 = np.clip((arr_xyz - vmin) * (255.0 / max(1e-6, (vmax - vmin))), 0, 255).astype(np.uint8)
del arr_zyx, arr_xyz; gc.collect()
log(f"Loaded membrane channel: shape={X,Y,Z} min/max={vmin:.2f}/{vmax:.2f} → 8-bit")

# (2) Preprocess: Gaussian → median (despeckle) → grey closing
prep = gaussian_filter(arr8.astype(np.float32), sigma=GAUSS_SIGMA)
prep = median_filter(prep, size=DESPECKLE_SIZE)
prep = grey_closing(prep, footprint=ball(GREY_CLOSE_RAD)).astype(np.float32)
log(f"Pre-clean: Gaussian sigma={GAUSS_SIGMA}, Despeckle={DESPECKLE_SIZE}, GreyClose rad={GREY_CLOSE_RAD}")

# (3) Fixed threshold
mask = (prep >= float(THRESH_VAL_U8))
del prep, arr8; gc.collect()
log(f"Threshold: value={THRESH_VAL_U8} → voxels={int(mask.sum())}")

# (4) 3D fill holes + opening
mask = binary_fill_holes(mask)
if OPEN_RAD_3D > 0:
    mask = binary_opening(mask, footprint=ball(int(OPEN_RAD_3D)))
log(f"Post-bin: FillHoles=3D, Opening rad={OPEN_RAD_3D} → voxels={int(mask.sum())}")

# (5) 3D connected components, keep ALL ≥ MIN_SIZE_VOX
labels3d, nlab = label(mask)
del mask; gc.collect()
if nlab == 0:
    raise RuntimeError("No objects after thresholding. Lower THRESH_VAL_U8 or adjust smoothing.")
counts = np.bincount(labels3d.ravel()); counts[0] = 0
keep_ids = np.where(counts >= MIN_SIZE_VOX)[0]
if keep_ids.size == 0:
    keep_ids = np.where(counts > 0)[0]  # if nothing passes the floor, keep everything nonzero
keep_set = set(int(i) for i in keep_ids)
log(f"3D CC: total={nlab}, kept={len(keep_ids)} (≥{MIN_SIZE_VOX} vox)")

# (6) Per-slice, per-lumen: smooth edge → **fill inside outer outline** (no circles)
filled = np.zeros((X, Y, Z), dtype=np.uint8)
se_close = disk(max(1, int(SL_CLOSE_RAD))) if SL_CLOSE_RAD > 0 else None
se_open  = disk(max(1, int(SL_OPEN_RAD)))  if SL_OPEN_RAD  > 0 else None

for z in range(Z):
    lab_z = labels3d[:, :, z]
    if lab_z.max() == 0:
        continue
    # process each kept lumen present in this slice
    present = np.unique(lab_z)
    for lid in present:
        if lid == 0 or lid not in keep_set:
            continue
        sl = (lab_z == lid)

        # Smooth the outer ring a bit (2D closing)
        if se_close is not None:
            sl = binary_closing(sl, footprint=se_close)

        # Remove tiny specks on the slice
        sl = remove_small_objects(sl, min_size=int(SL_MIN_OBJECT_2D))

        # **Fill interior** of the ring on this slice (this is the key difference vs circles)
        sl = binary_fill_holes(sl)

        # Remove small internal holes that may remain
        sl = remove_small_holes(sl, area_threshold=int(SL_HOLE_AREA_2D))

        # Optional 2D opening to trim thin protrusions
        if se_open is not None:
            sl = binary_opening(sl, footprint=se_open)

        filled[:, :, z] |= sl.astype(np.uint8)

del labels3d; gc.collect()
log("Slice-wise fill-inside-outline: done for all kept lumens.")

# (7) Gentle 3D smoothing for Z continuity (no heavy SDF)
bin_mask = filled.astype(bool)
if FINAL_CLOSE_RAD > 0:
    bin_mask = binary_closing(bin_mask, footprint=ball(int(FINAL_CLOSE_RAD)))
if FINAL_OPEN_RAD > 0:
    bin_mask = binary_opening(bin_mask, footprint=ball(int(FINAL_OPEN_RAD)))
final_mask = bin_mask.astype(np.uint8)
del bin_mask, filled; gc.collect()

# (8) Save + auto-download
out_path = os.path.join(OUTPUT_DIR, OUT_TIF)
sitk.WriteImage(to_itk_u8(final_mask * 255), out_path, True)
log(f"Saved: {out_path}")
log(f"Done in {time.time()-t0:.2f}s → {OUTPUT_DIR}")

# Auto-download in Colab (ignored elsewhere)
try:
    from google.colab import files
    files.download(out_path)
except Exception as e:
    log(f"Auto-download skipped (not Colab or files API unavailable): {e}")


[LOG] Loaded membrane channel: shape=(1024, 1024, 117) min/max=0.00/255.00 → 8-bit
[LOG] Pre-clean: Gaussian sigma=(1.2, 1.2, 1.2), Despeckle=3, GreyClose rad=1
[LOG] Threshold: value=26 → voxels=2576261
[LOG] Post-bin: FillHoles=3D, Opening rad=2 → voxels=1600849
[LOG] 3D CC: total=72, kept=3 (≥5000 vox)
[LOG] Slice-wise fill-inside-outline: done for all kept lumens.
[LOG] Saved: /content/out_lumen3d/lumen_mask_3d.tif
[LOG] Done in 221.87s → /content/out_lumen3d


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# --- Download only the lumen segmentation 3D images ---
import shutil
from google.colab import files  # works in Colab

OUTPUT_DIR = "/content/out_lumen3d"  # same as in your pipeline

# Option 1: download the main filled lumen mask
files.download(f"{OUTPUT_DIR}/lumen_mask_3d.tif")

# Option 2: (optional) download the per-slice outer contour image
files.download(f"{OUTPUT_DIR}/lumen_outline_3d.tif")

# Option 3: (optional) zip everything (mask + outline + CSV if present)
shutil.make_archive("/content/lumen3d_all_outputs", "zip", OUTPUT_DIR)
files.download("/content/lumen3d_all_outputs.zip")


In [ ]:
# import shutil, os
# zip_path = f"/content/{base}_nnInteractive.zip"
# shutil.make_archive(zip_path[:-4], "zip", f"/content/{base}_nnInteractive")
# from google.colab import files
# files.download(zip_path)